In [10]:
!pip install qiskit qiskit_aer


In [11]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import numpy as np
import matplotlib.pyplot as plt


In [12]:
def qft_rotations(qc, n):
    """Apply QFT rotations recursively on the first n qubits."""
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc


In [13]:
def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc


In [14]:
def qft_circuit(n):
    """Builds an n-qubit Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc


In [15]:
def run_qft(n):
    """Executes QFT circuit and displays Bloch sphere representation."""
    qc = qft_circuit(n)

    # Create a simulator
    simulator = AerSimulator(method='statevector')
    qc.save_statevector()

    # Transpile and run
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()

    # Retrieve statevector
    statevector = result.data(0)["statevector"]

    # Display results
    plot_bloch_multivector(statevector)
    plt.show()
    print(qc.draw(output='text'))


In [16]:
for n_qubits in [2, 3, 4]:
    print(f"\nRunning Quantum Fourier Transform on {n_qubits} qubits...")
    run_qft(n_qubits)



Running Quantum Fourier Transform on 2 qubits...
                   ┌───┐    statevector 
q_0: ──────■───────┤ H ├─X───────░──────
     ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ┤ H ├─■─────────────X───────░──────
     └───┘                       ░      

Running Quantum Fourier Transform on 3 qubits...
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                              ░      

Running Quantum Fourier Transform on 4 qubits...
                                                                          ┌───┐»
q_0: ──────■───────────────────────────────■──────────────────────■───────┤ H ├»
           │                       

In [17]:
def inverse_qft(n):
    """Build and return an inverse QFT circuit."""
    qc = QuantumCircuit(n)
    swap_registers(qc, n)
    for qubit in range(n):
        for m in range(qubit):
            qc.cp(-np.pi / 2**(qubit - m), m, qubit)
        qc.h(qubit)
    swap_registers(qc, n)
    return qc

# Run inverse QFT for 3 qubits
inv_qc = inverse_qft(3)
print("\nInverse QFT Circuit:")
print(inv_qc.draw(output='text'))



Inverse QFT Circuit:
        ┌───┐                                           
q_0: ─X─┤ H ├─■──────────────■────────────────────────X─
      │ └───┘ │P(-π/2) ┌───┐ │                        │ 
q_1: ─┼───────■────────┤ H ├─┼─────────■──────────────┼─
      │                └───┘ │P(-π/4)  │P(-π/2) ┌───┐ │ 
q_2: ─X──────────────────────■─────────■────────┤ H ├─X─
                                                └───┘   


In [18]:
def measure_and_simulate(n):
    """Run QFT with measurement and show probability distribution."""
    qc = qft_circuit(n)
    qc.measure_all()
    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1024).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()

# Run for 3 qubits
print("\nRunning measured QFT for 3 qubits...")
measure_and_simulate(3)



Running measured QFT for 3 qubits...
